## **Introduction**

Fraud detection in the banking sector is a critical endeavor, aiming to safeguard both financial institutions and customers from malicious activities. In this post, we will explore a practical approach to combine fraud datasets from a bank with a clickstream dataset to enhance our understanding of user behavior and improve fraud detection strategies.

## **Import Libraries**

In [1]:
import numpy as np
import pandas as pd 
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

## **Dataset Overview**

We possess one training dataset, one testing (unseen) dataset, and a clickstream dataset, all interconnected through a common identifier known as "SESSION_ID." This identifier allows us to link user actions across these datasets. A session involves client online banking activities like signing in, updating passwords, viewing products, or adding items to the cart.

Majority of fraud cases add new shipping address, or change password. That's why we are combining these datasets to enhance the fraud strategy. Additionally, combining the datasets simplifies analysis and reduces computational load. I'll cover this part in the EDA section.

For more information about the dataset or to use it for your fraud prediction projects, please refer to [**Fraud detection in Banks**](https://www.kaggle.com/datasets/mohammadbolandraftar/my-dataset) dataset.

In [113]:
clickstream_dataset = pd.read_csv("/kaggle/input/my-dataset/Clickstream dataset.csv")

train_dataset = pd.read_csv("/kaggle/input/my-dataset/Transaction dataset train set.csv")
unseen_dataset = pd.read_csv("/kaggle/input/my-dataset/Transaction dataset test set.csv")

print("clickstream_dataset size:", clickstream_dataset.shape)
print("train_dataset size:", train_dataset.shape)
print("test_dataset size:", unseen_dataset.shape)

clickstream_dataset size: (408837, 10)
train_dataset size: (11903, 146)
test_dataset size: (2976, 146)


In [114]:
clickstream_df = clickstream_dataset.copy()
train_df = train_dataset.copy()
unseen_df = test_dataset.copy()

Our training dataset has 146 columns and 11903 rows. Our target column is FRAUD_FLAG. So, for prediction purposes, we'll have a binary classification case. 

In [115]:
train_df.head(2)

,SESSION_ID,EVENT_DATETIME,TRANSACTION_ID,FRAUD_FLAG,AVAIL_CRDT,AMOUNT,CREDIT_LIMIT,CARD_NOT_PRESENT,FLAG_LX,FLAG_ATM,FLAG_AUTO,FLAG_CASH,FLAG_APPAREL,FLAG_DEAL,FLAG_RECREA,FLAG_ENTERTAINMENT,FLAG_GIFT_CARD,FLAG_FASTFOOD,FLAG_GAS,FLAG_HIGH_AMT,FLAG_HIGH_RECREA,FLAG_INTERNET,FLAG_INTERNATIONAL,FLAG_FASHION,FLAG_LOW_AMT,FLAG_MANUAL_ENTRY,FLAG_ELCTRNCS,FLAG_SWIPE,FLAG_HEALTH,MEAN_AUTO_PAST_7DAY,MEAN_APPAREL_PAST_7DAY,MEAN_RECREA_PAST_7DAY,MEAN_GIFT_CARD_PAST_7DAY,MEAN_FASTFOOD_PAST_7DAY,MEAN_HIGH_AMT_PAST_7DAY,MEAN_HIGH_RECREA_PAST_7DAY,MEAN_INTERNET_PAST_7DAY,MEAN_INTERNATIONAL_PAST_7DAY,MEAN_FASHION_PAST_7DAY,MEAN_LOW_AMT_PAST_7DAY,MEAN_MANUAL_ENTRY_PAST_7DAY,MEAN_ELCTRNCS_PAST_7DAY,MEAN_SWIPE_PAST_7DAY,MEAN_HEALTH_PAST_7DAY,MEAN_WEEKEND_PAST_7DAY,MAX_CASH_PAST_7DAY,MAX_APPAREL_PAST_7DAY,MAX_RECREA_PAST_7DAY,MAX_HIGH_AMT_PAST_7DAY,MAX_HIGH_RECREA_PAST_7DAY,MAX_INTERNET_PAST_7DAY,MAX_SWIPE_PAST_7DAY,MAX_WEEKEND_PAST_7DAY,STD_LX_PAST_7DAY,STD_FASTFOOD_PAST_7DAY,STD_HIGH_AMT_PAST_7DAY,STD_INTERNET_PAST_7DAY,STD_LOW_AMT_PAST_7DAY,STD_SWIPE_PAST_7DAY,STD_HEALTH_PAST_7DAY,SUM_LX_PAST_7DAY,SUM_AUTO_PAST_7DAY,SUM_APPAREL_PAST_7DAY,SUM_RECREA_PAST_7DAY,SUM_GAS_PAST_7DAY,SUM_HIGH_AMT_PAST_7DAY,SUM_INTERNET_PAST_7DAY,SUM_INTERNATIONAL_PAST_7DAY,SUM_LOW_AMT_PAST_7DAY,SUM_MANUAL_ENTRY_PAST_7DAY,SUM_PARTIAL_PAST_7DAY,SUM_ELCTRNCS_PAST_7DAY,SUM_SWIPE_PAST_7DAY,SUM_WEEKEND_PAST_7DAY,COUNT_AUTO_PAST_7DAY,COUNT_ENTERTAINMENT_PAST_7DAY,COUNT_GAS_PAST_7DAY,COUNT_HIGH_AMT_PAST_7DAY,COUNT_INTERNET_PAST_7DAY,COUNT_LOW_AMT_PAST_7DAY,COUNT_MANUAL_ENTRY_PAST_7DAY,COUNT_SWIPE_PAST_7DAY,COUNT_HEALTH_PAST_7DAY,COUNT_WEEKEND_PAST_7DAY,MEAN_AUTO_PAST_30DAY,MEAN_DEAL_PAST_30DAY,MEAN_RECREA_PAST_30DAY,MEAN_ENTERTAINMENT_PAST_30DAY,MEAN_GIFT_CARD_PAST_30DAY,MEAN_HIGH_AMT_PAST_30DAY,MEAN_INTERNET_PAST_30DAY,MEAN_LOW_AMT_PAST_30DAY,MEAN_MANUAL_ENTRY_PAST_30DAY,MEAN_ELCTRNCS_PAST_30DAY,MEAN_SWIPE_PAST_30DAY,MEAN_HEALTH_PAST_30DAY,MEAN_WEEKEND_PAST_30DAY,MAX_AUTO_PAST_30DAY,MAX_APPAREL_PAST_30DAY,MAX_ENTERTAINMENT_PAST_30DAY,MAX_FASTFOOD_PAST_30DAY,MAX_HIGH_RECREA_PAST_30DAY,MAX_MANUAL_ENTRY_PAST_30DAY,MAX_PARTIAL_PAST_30DAY,MAX_WEEKEND_PAST_30DAY,STD_AUTO_PAST_30DAY,STD_APPAREL_PAST_30DAY,STD_RECREA_PAST_30DAY,STD_ENTERTAINMENT_PAST_30DAY,STD_GIFT_CARD_PAST_30DAY,STD_HIGH_RECREA_PAST_30DAY,STD_INTERNET_PAST_30DAY,STD_LOW_AMT_PAST_30DAY,STD_MANUAL_ENTRY_PAST_30DAY,STD_PARTIAL_PAST_30DAY,STD_SWIPE_PAST_30DAY,STD_HEALTH_PAST_30DAY,SUM_AUTO_PAST_30DAY,SUM_APPAREL_PAST_30DAY,SUM_DEAL_PAST_30DAY,SUM_RECREA_PAST_30DAY,SUM_ENTERTAINMENT_PAST_30DAY,SUM_GIFT_CARD_PAST_30DAY,SUM_FASTFOOD_PAST_30DAY,SUM_GAS_PAST_30DAY,SUM_HIGH_AMT_PAST_30DAY,SUM_HIGH_RECREA_PAST_30DAY,SUM_INTERNET_PAST_30DAY,SUM_INTERNATIONAL_PAST_30DAY,SUM_LOW_AMT_PAST_30DAY,SUM_MANUAL_ENTRY_PAST_30DAY,SUM_SWIPE_PAST_30DAY,SUM_WEEKEND_PAST_30DAY,COUNT_AUTO_PAST_30DAY,COUNT_RECREA_PAST_30DAY,COUNT_GIFT_CARD_PAST_30DAY,COUNT_FASTFOOD_PAST_30DAY,COUNT_GAS_PAST_30DAY,COUNT_HIGH_AMT_PAST_30DAY,COUNT_INTERNET_PAST_30DAY,COUNT_LOW_AMT_PAST_30DAY,COUNT_MANUAL_ENTRY_PAST_30DAY,COUNT_ELCTRNCS_PAST_30DAY,COUNT_SWIPE_PAST_30DAY,COUNT_HEALTH_PAST_30DAY,COUNT_WEEKEND_PAST_30DAY
0,9cc2c261-b73e-4f73-94c7-cc5d78e01148,2022-03-02 15:40:49,71f8a325e,1.0,7420.73,69.70,14600.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,341.25,358.75,0.0,358.75,0.0,26.32,0.0,0.0,0.00,358.75,0.0,15.60,0.0,0.00,0.0,0.0,358.75,341.25,0.0,26.32,15.60,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,358.75,0.00,358.75,27.68,0.0,0.00,358.75,0.0,0.0,15.6,0.00,0,0,0,0,1,0,0,1,0,0,0.0,0.00,358.75,0.0,341.25,341.25,26.32,0.00,341.25,0.0,16.40,0.0,0.0,0.0,0.00,0.0,0.0,0.0,358.75,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,341.25,0.0,358.75,0.0,0.00,341.25,0.0,27.68,0.0,0.00,341.25,15.60,0.0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,f3fc1ca4-5e08-42d8-bf5d-bc5026a377b5,2022-07-04 07:51:19,2b4a3e059,0.0,3825.90,1.02,7300.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,1.02,18.52,0.0,12.35,0.0,

Our unseen dataset is the same as the train_df, except the fact that the "FRAUD_FLAG" column is empty and we need to predict that. 

In [116]:
unseen_df.head(2)

,SESSION_ID,EVENT_DATETIME,TRANSACTION_ID,FRAUD_FLAG,AVAIL_CRDT,AMOUNT,CREDIT_LIMIT,CARD_NOT_PRESENT,FLAG_LX,FLAG_ATM,FLAG_AUTO,FLAG_CASH,FLAG_APPAREL,FLAG_DEAL,FLAG_RECREA,FLAG_ENTERTAINMENT,FLAG_GIFT_CARD,FLAG_FASTFOOD,FLAG_GAS,FLAG_HIGH_AMT,FLAG_HIGH_RECREA,FLAG_INTERNET,FLAG_INTERNATIONAL,FLAG_FASHION,FLAG_LOW_AMT,FLAG_MANUAL_ENTRY,FLAG_ELCTRNCS,FLAG_SWIPE,FLAG_HEALTH,MEAN_AUTO_PAST_7DAY,MEAN_APPAREL_PAST_7DAY,MEAN_RECREA_PAST_7DAY,MEAN_GIFT_CARD_PAST_7DAY,MEAN_FASTFOOD_PAST_7DAY,MEAN_HIGH_AMT_PAST_7DAY,MEAN_HIGH_RECREA_PAST_7DAY,MEAN_INTERNET_PAST_7DAY,MEAN_INTERNATIONAL_PAST_7DAY,MEAN_FASHION_PAST_7DAY,MEAN_LOW_AMT_PAST_7DAY,MEAN_MANUAL_ENTRY_PAST_7DAY,MEAN_ELCTRNCS_PAST_7DAY,MEAN_SWIPE_PAST_7DAY,MEAN_HEALTH_PAST_7DAY,MEAN_WEEKEND_PAST_7DAY,MAX_CASH_PAST_7DAY,MAX_APPAREL_PAST_7DAY,MAX_RECREA_PAST_7DAY,MAX_HIGH_AMT_PAST_7DAY,MAX_HIGH_RECREA_PAST_7DAY,MAX_INTERNET_PAST_7DAY,MAX_SWIPE_PAST_7DAY,MAX_WEEKEND_PAST_7DAY,STD_LX_PAST_7DAY,STD_FASTFOOD_PAST_7DAY,STD_HIGH_AMT_PAST_7DAY,STD_INTERNET_PAST_7DAY,STD_LOW_AMT_PAST_7DAY,STD_SWIPE_PAST_7DAY,STD_HEALTH_PAST_7DAY,SUM_LX_PAST_7DAY,SUM_AUTO_PAST_7DAY,SUM_APPAREL_PAST_7DAY,SUM_RECREA_PAST_7DAY,SUM_GAS_PAST_7DAY,SUM_HIGH_AMT_PAST_7DAY,SUM_INTERNET_PAST_7DAY,SUM_INTERNATIONAL_PAST_7DAY,SUM_LOW_AMT_PAST_7DAY,SUM_MANUAL_ENTRY_PAST_7DAY,SUM_PARTIAL_PAST_7DAY,SUM_ELCTRNCS_PAST_7DAY,SUM_SWIPE_PAST_7DAY,SUM_WEEKEND_PAST_7DAY,COUNT_AUTO_PAST_7DAY,COUNT_ENTERTAINMENT_PAST_7DAY,COUNT_GAS_PAST_7DAY,COUNT_HIGH_AMT_PAST_7DAY,COUNT_INTERNET_PAST_7DAY,COUNT_LOW_AMT_PAST_7DAY,COUNT_MANUAL_ENTRY_PAST_7DAY,COUNT_SWIPE_PAST_7DAY,COUNT_HEALTH_PAST_7DAY,COUNT_WEEKEND_PAST_7DAY,MEAN_AUTO_PAST_30DAY,MEAN_DEAL_PAST_30DAY,MEAN_RECREA_PAST_30DAY,MEAN_ENTERTAINMENT_PAST_30DAY,MEAN_GIFT_CARD_PAST_30DAY,MEAN_HIGH_AMT_PAST_30DAY,MEAN_INTERNET_PAST_30DAY,MEAN_LOW_AMT_PAST_30DAY,MEAN_MANUAL_ENTRY_PAST_30DAY,MEAN_ELCTRNCS_PAST_30DAY,MEAN_SWIPE_PAST_30DAY,MEAN_HEALTH_PAST_30DAY,MEAN_WEEKEND_PAST_30DAY,MAX_AUTO_PAST_30DAY,MAX_APPAREL_PAST_30DAY,MAX_ENTERTAINMENT_PAST_30DAY,MAX_FASTFOOD_PAST_30DAY,MAX_HIGH_RECREA_PAST_30DAY,MAX_MANUAL_ENTRY_PAST_30DAY,MAX_PARTIAL_PAST_30DAY,MAX_WEEKEND_PAST_30DAY,STD_AUTO_PAST_30DAY,STD_APPAREL_PAST_30DAY,STD_RECREA_PAST_30DAY,STD_ENTERTAINMENT_PAST_30DAY,STD_GIFT_CARD_PAST_30DAY,STD_HIGH_RECREA_PAST_30DAY,STD_INTERNET_PAST_30DAY,STD_LOW_AMT_PAST_30DAY,STD_MANUAL_ENTRY_PAST_30DAY,STD_PARTIAL_PAST_30DAY,STD_SWIPE_PAST_30DAY,STD_HEALTH_PAST_30DAY,SUM_AUTO_PAST_30DAY,SUM_APPAREL_PAST_30DAY,SUM_DEAL_PAST_30DAY,SUM_RECREA_PAST_30DAY,SUM_ENTERTAINMENT_PAST_30DAY,SUM_GIFT_CARD_PAST_30DAY,SUM_FASTFOOD_PAST_30DAY,SUM_GAS_PAST_30DAY,SUM_HIGH_AMT_PAST_30DAY,SUM_HIGH_RECREA_PAST_30DAY,SUM_INTERNET_PAST_30DAY,SUM_INTERNATIONAL_PAST_30DAY,SUM_LOW_AMT_PAST_30DAY,SUM_MANUAL_ENTRY_PAST_30DAY,SUM_SWIPE_PAST_30DAY,SUM_WEEKEND_PAST_30DAY,COUNT_AUTO_PAST_30DAY,COUNT_RECREA_PAST_30DAY,COUNT_GIFT_CARD_PAST_30DAY,COUNT_FASTFOOD_PAST_30DAY,COUNT_GAS_PAST_30DAY,COUNT_HIGH_AMT_PAST_30DAY,COUNT_INTERNET_PAST_30DAY,COUNT_LOW_AMT_PAST_30DAY,COUNT_MANUAL_ENTRY_PAST_30DAY,COUNT_ELCTRNCS_PAST_30DAY,COUNT_SWIPE_PAST_30DAY,COUNT_HEALTH_PAST_30DAY,COUNT_WEEKEND_PAST_30DAY
0,2810bffa-ac9d-4dc5-9cfa-369a93e6a1f1,2022-04-05 15:12:58,8b4b6f1f5,NaN,2074.60,1.02,7300.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0.0,0.00,9.22,0.00,0.00,0.00,0.00,50.23,0.0,0.0,8.78,9.22,0.0,0.00,0.0,0.00,0.0,0.0,9.22,0.0,0.00,47.77,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,9.22,0.00,0.00,50.23,0.0,8.78,8.78,0.0,0.0,0.00,0.0,0,0,0,0,0,0,0,0,0,0,45.1,0.00,27.79,0.00,0.00,154.78,47.77,9.22,22.55,0.00,73.37,0.00,101.89,42.9,0.0,0.00,0.0,0.00,35.88,0.0,154.78,0.0,0.00,28.27,0.00,0.00,0.00,0.00,0.00,17.93,0.0,52.11,0.00,45.1,0.0,0.00,55.58,0.00,0.0,0.00,0.00,147.22,0.0,100.45,0.0,8.78,42.90,293.48,214.22,0,1,0,0,0,1,2,1,2,0,4,0,2
1,cc7ad152-ebf9-4399-8d33-fe1442ae20f9,2022-04-19 01:22:51,7530264ab,NaN,25250.88,0.97,29200.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0.0,20.15,24.50,121.88,3.08,182.96,18.96,1.02,0.0,0.0,4.10,0.00,0.0,39.

As mentioned, the "clickstream_df" contains the user online banking activities. It also contains the time user logged in/out, their browser/OS/device information. In addition, it includes the location of the users.

In [117]:
clickstream_df.head()

,SESSION_ID,TIMESTAMP,PAGE_NAME,BROWSER_FAMILY,BROWSER_VERSION,OS_FAMILY,DEVICE_FAMILY,DEVICE_BRAND,DEVICE_MODEL,CITY
0,1f92304e-dfc1-43df-9651-fa8b423ff494,2022-02-01 12:51:22,sign-in,Mobile Safari UI/WKWebView,NaN,iOS,iPhone,Apple,iPhone,lasalle
1,1f92304e-dfc1-43df-9651-fa8b423ff494,2022-02-01 12:51:30,home-page,Mobile Safari UI/WKWebView,NaN,iOS,iPhone,Apple,iPhone,lasalle
2,1f92304e-dfc1-43df-9651-fa8b423ff494,2022-02-01 12:51:53,account-settings,Mobile Safari UI/WKWebView,NaN,iOS,iPhone,Apple,iPhone,lasalle
3,1f92304e-dfc1-43df-9651-fa8b423ff494,2022-02-01 12:52:02,add-payment-method,Mobile Safari UI/WKWebView,NaN,iOS,iPhone,Apple,iPhone,lasalle
4,1f92304e-dfc1-43df-9651-fa8b423ff494,2022-02-01 12:52:41,checkout,Mobile Safari UI/WKWebView,NaN,iOS,iPhone,Apple,iPhone,lasalle


## **Check the SESSION ID**

As you see, the first 70 rows of the "SESSION_ID" column in the clickstream_df belong to one user, as all the rows from 0 to 69 have the same "SESSION_ID"

In [118]:
clickstream_df["SESSION_ID"][0:71]

0     1f92304e-dfc1-43df-9651-fa8b423ff494
1     1f92304e-dfc1-43df-9651-fa8b423ff494
2     1f92304e-dfc1-43df-9651-fa8b423ff494
3     1f92304e-dfc1-43df-9651-fa8b423ff494
4     1f92304e-dfc1-43df-9651-fa8b423ff494
                      ...                 
66    1f92304e-dfc1-43df-9651-fa8b423ff494
67    1f92304e-dfc1-43df-9651-fa8b423ff494
68    1f92304e-dfc1-43df-9651-fa8b423ff494
69    1f92304e-dfc1-43df-9651-fa8b423ff494
70    1c9dae34-74d1-4842-8446-e468e27d340f
Name: SESSION_ID, Length: 71, dtype: object

## **Data Types**

Since we will use this dataset for fraud detection purposes, we need to make sure that the data types are correct.

In [119]:
clickstream_df.dtypes

SESSION_ID         object
TIMESTAMP          object
PAGE_NAME          object
BROWSER_FAMILY     object
BROWSER_VERSION    object
OS_FAMILY          object
DEVICE_FAMILY      object
DEVICE_BRAND       object
DEVICE_MODEL       object
CITY               object
dtype: object

The "TIMESTAMP" records the date/time of each event. However, it's an object. Let's change its type to date/time.

In [120]:
clickstream_df["TIMESTAMP"] = pd.to_datetime(clickstream_df["TIMESTAMP"])

## **Filter by SESSION_ID**

To combine the datasets, we can do it in 2 ways: 
* Create a dictionary of unique events and the number of each event
* Create the unique events happened in each session. In Python, we can do this through "set" data structure.

We will apply the second one in this example. 

In addition, we will record the begining and end time of each SESSION_ID through two coulmns, "TIMESTAMP" [min, max].

In [121]:
aggregated_clickstream = clickstream_df.groupby(by=["SESSION_ID"]).agg({
    "TIMESTAMP": ["min", "max"], 
    "PAGE_NAME": (lambda x: set(x)), # to have unique occurrence of the events
    "BROWSER_FAMILY": "first",
    "BROWSER_VERSION": "first",
    "OS_FAMILY": "first",
    "DEVICE_FAMILY": "first",
    "DEVICE_BRAND": "first",
    "DEVICE_MODEL": "first",
    "CITY": "first"   
})

One of the ways to deal with date/time variables is to calculate the time they spend on online banking. The time of the day they login to their account, could be another important feature in fraud detection (we'll cover them in the EDA part). 

I'm creating a new feature "TIME_SPENT" which contains the time of each session in seconds. To calculate the time gap, we subtract the  TIMESTAMP["min"] from TIMESTAMP["max"].

In [122]:
aggregated_clickstream["TIME_SPENT"] = (aggregated_clickstream["TIMESTAMP"]["max"] - 
                                        aggregated_clickstream["TIMESTAMP"]["min"]).dt.total_seconds()

Let's check the length of the datasets

In [123]:
print("aggregated_clickstream size:", len(aggregated_clickstream))

print("length of sum of train and unssen datasets:", len(unseen_df) + len(train_df))

aggregated_clickstream size: 14879
length of sum of train and unssen datasets: 14879


It seems that our code worked!

In [124]:
aggregated_clickstream.columns = ["TIMESTAMP_MIN", "TIMESTAMP_MAX", "PAGE_NAME", 
                                  "BROWSER_FAMILY", "BROWSER_VERSION", "OS_FAMILY", "DEVICE_FAMILY", 
                                  "DEVICE_BRAND", "DEVICE_MODEL", "CITY", "TIME_SPENT"
                                 ]

In [125]:
aggregated_clickstream.reset_index(inplace=True)

In [126]:
aggregated_clickstream.head(2)

,SESSION_ID,TIMESTAMP_MIN,TIMESTAMP_MAX,PAGE_NAME,BROWSER_FAMILY,BROWSER_VERSION,OS_FAMILY,DEVICE_FAMILY,DEVICE_BRAND,DEVICE_MODEL,CITY,TIME_SPENT
0,00019a29-2653-4bef-8546-f3a406631cf1,2022-05-13 21:51:24,2022-05-13 22:01:43,"{view-product, checkout, update-payment-method, view-cart, add-to-cart, payment-failed, search-results, order-complete, sign-in, order-history, home-page}",Android,10,Android,Motorola e,Motorola,e,north york,619.0
1,00074eae-212b-4df4-919b-bda4cd329108,2022-05-07 04:23:02,2022-05-07 04:31:28,"{view-product, checkout, update-payment-method, view-cart, add-to-cart, payment-failed, search-results, order-complete, sign-in, home-page}",Android,11,Android,Samsung SM-A505W,Samsung,SM-A505W,montreal,506.0


As it's seen, the "PAGE_NAME" column contains all the user activities during each session.

## **Merge Datasets**

In [127]:
train_df = pd.merge(train_df, aggregated_clickstream, on="SESSION_ID", how="left")
unseen_df = pd.merge(unseen_df, aggregated_clickstream, on= "SESSION_ID", how="left")

In [128]:
train_df.head(2)

,SESSION_ID,EVENT_DATETIME,TRANSACTION_ID,FRAUD_FLAG,AVAIL_CRDT,AMOUNT,CREDIT_LIMIT,CARD_NOT_PRESENT,FLAG_LX,FLAG_ATM,FLAG_AUTO,FLAG_CASH,FLAG_APPAREL,FLAG_DEAL,FLAG_RECREA,FLAG_ENTERTAINMENT,FLAG_GIFT_CARD,FLAG_FASTFOOD,FLAG_GAS,FLAG_HIGH_AMT,FLAG_HIGH_RECREA,FLAG_INTERNET,FLAG_INTERNATIONAL,FLAG_FASHION,FLAG_LOW_AMT,FLAG_MANUAL_ENTRY,FLAG_ELCTRNCS,FLAG_SWIPE,FLAG_HEALTH,MEAN_AUTO_PAST_7DAY,MEAN_APPAREL_PAST_7DAY,MEAN_RECREA_PAST_7DAY,MEAN_GIFT_CARD_PAST_7DAY,MEAN_FASTFOOD_PAST_7DAY,MEAN_HIGH_AMT_PAST_7DAY,MEAN_HIGH_RECREA_PAST_7DAY,MEAN_INTERNET_PAST_7DAY,MEAN_INTERNATIONAL_PAST_7DAY,MEAN_FASHION_PAST_7DAY,MEAN_LOW_AMT_PAST_7DAY,MEAN_MANUAL_ENTRY_PAST_7DAY,MEAN_ELCTRNCS_PAST_7DAY,MEAN_SWIPE_PAST_7DAY,MEAN_HEALTH_PAST_7DAY,MEAN_WEEKEND_PAST_7DAY,MAX_CASH_PAST_7DAY,MAX_APPAREL_PAST_7DAY,MAX_RECREA_PAST_7DAY,MAX_HIGH_AMT_PAST_7DAY,MAX_HIGH_RECREA_PAST_7DAY,MAX_INTERNET_PAST_7DAY,MAX_SWIPE_PAST_7DAY,MAX_WEEKEND_PAST_7DAY,STD_LX_PAST_7DAY,STD_FASTFOOD_PAST_7DAY,STD_HIGH_AMT_PAST_7DAY,STD_INTERNET_PAST_7DAY,STD_LOW_AMT_PAST_7DAY,STD_SWIPE_PAST_7DAY,STD_HEALTH_PAST_7DAY,SUM_LX_PAST_7DAY,SUM_AUTO_PAST_7DAY,SUM_APPAREL_PAST_7DAY,SUM_RECREA_PAST_7DAY,SUM_GAS_PAST_7DAY,SUM_HIGH_AMT_PAST_7DAY,SUM_INTERNET_PAST_7DAY,SUM_INTERNATIONAL_PAST_7DAY,SUM_LOW_AMT_PAST_7DAY,SUM_MANUAL_ENTRY_PAST_7DAY,SUM_PARTIAL_PAST_7DAY,SUM_ELCTRNCS_PAST_7DAY,SUM_SWIPE_PAST_7DAY,SUM_WEEKEND_PAST_7DAY,COUNT_AUTO_PAST_7DAY,COUNT_ENTERTAINMENT_PAST_7DAY,COUNT_GAS_PAST_7DAY,COUNT_HIGH_AMT_PAST_7DAY,COUNT_INTERNET_PAST_7DAY,COUNT_LOW_AMT_PAST_7DAY,COUNT_MANUAL_ENTRY_PAST_7DAY,COUNT_SWIPE_PAST_7DAY,COUNT_HEALTH_PAST_7DAY,COUNT_WEEKEND_PAST_7DAY,MEAN_AUTO_PAST_30DAY,MEAN_DEAL_PAST_30DAY,MEAN_RECREA_PAST_30DAY,MEAN_ENTERTAINMENT_PAST_30DAY,MEAN_GIFT_CARD_PAST_30DAY,MEAN_HIGH_AMT_PAST_30DAY,MEAN_INTERNET_PAST_30DAY,MEAN_LOW_AMT_PAST_30DAY,MEAN_MANUAL_ENTRY_PAST_30DAY,MEAN_ELCTRNCS_PAST_30DAY,MEAN_SWIPE_PAST_30DAY,MEAN_HEALTH_PAST_30DAY,MEAN_WEEKEND_PAST_30DAY,MAX_AUTO_PAST_30DAY,MAX_APPAREL_PAST_30DAY,MAX_ENTERTAINMENT_PAST_30DAY,MAX_FASTFOOD_PAST_30DAY,MAX_HIGH_RECREA_PAST_30DAY,MAX_MANUAL_ENTRY_PAST_30DAY,MAX_PARTIAL_PAST_30DAY,MAX_WEEKEND_PAST_30DAY,STD_AUTO_PAST_30DAY,STD_APPAREL_PAST_30DAY,STD_RECREA_PAST_30DAY,STD_ENTERTAINMENT_PAST_30DAY,STD_GIFT_CARD_PAST_30DAY,STD_HIGH_RECREA_PAST_30DAY,STD_INTERNET_PAST_30DAY,STD_LOW_AMT_PAST_30DAY,STD_MANUAL_ENTRY_PAST_30DAY,STD_PARTIAL_PAST_30DAY,STD_SWIPE_PAST_30DAY,STD_HEALTH_PAST_30DAY,SUM_AUTO_PAST_30DAY,SUM_APPAREL_PAST_30DAY,SUM_DEAL_PAST_30DAY,SUM_RECREA_PAST_30DAY,SUM_ENTERTAINMENT_PAST_30DAY,SUM_GIFT_CARD_PAST_30DAY,SUM_FASTFOOD_PAST_30DAY,SUM_GAS_PAST_30DAY,SUM_HIGH_AMT_PAST_30DAY,SUM_HIGH_RECREA_PAST_30DAY,SUM_INTERNET_PAST_30DAY,SUM_INTERNATIONAL_PAST_30DAY,SUM_LOW_AMT_PAST_30DAY,SUM_MANUAL_ENTRY_PAST_30DAY,SUM_SWIPE_PAST_30DAY,SUM_WEEKEND_PAST_30DAY,COUNT_AUTO_PAST_30DAY,COUNT_RECREA_PAST_30DAY,COUNT_GIFT_CARD_PAST_30DAY,COUNT_FASTFOOD_PAST_30DAY,COUNT_GAS_PAST_30DAY,COUNT_HIGH_AMT_PAST_30DAY,COUNT_INTERNET_PAST_30DAY,COUNT_LOW_AMT_PAST_30DAY,COUNT_MANUAL_ENTRY_PAST_30DAY,COUNT_ELCTRNCS_PAST_30DAY,COUNT_SWIPE_PAST_30DAY,COUNT_HEALTH_PAST_30DAY,COUNT_WEEKEND_PAST_30DAY,TIMESTAMP_MIN,TIMESTAMP_MAX,PAGE_NAME,BROWSER_FAMILY,BROWSER_VERSION,OS_FAMILY,DEVICE_FAMILY,DEVICE_BRAND,DEVICE_MODEL,CITY,TIME_SPENT
0,9cc2c261-b73e-4f73-94c7-cc5d78e01148,2022-03-02 15:40:49,71f8a325e,1.0,7420.73,69.70,14600.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,341.25,358.75,0.0,358.75,0.0,26.32,0.0,0.0,0.00,358.75,0.0,15.60,0.0,0.00,0.0,0.0,358.75,341.25,0.0,26.32,15.60,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,358.75,0.00,358.75,27.68,0.0,0.00,358.75,0.0,0.0,15.6,0.00,0,0,0,0,1,0,0,1,0,0,0.0,0.00,358.75,0.0,341.25,341.25,26.32,0.00,341.25,0.0,16.40,0.0,0.0,0.0,0.00,0.0,0.0,0.0,358.75,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,341.25,0.0,358.75,0.0,0.00,341.25,0.0,27.68,0.0,0.00,341.25,15.60,0.0,0,1,0,0,0,0,0,0,0,0,1,0,0,2022-03-02 15:40:49,2022-03-02 15:47:22,"{view-product, checkout, view-

In [130]:
print("train_df size:", train_df.shape)
print("test_df size:", unseen_df.shape)

train_df size: (11903, 157)
test_df size: (2976, 157)


## **Save Combined Datasets**

In [131]:
train_df.to_csv("train_dataset_combined.csv", index=False)
unseen_df.to_csv("test_dataset_combined.csv", index=False)

## **Conclusion**

The combination of fraud datasets from a bank with clickstream data offers a holistic perspective on user behavior. This approach enhances fraud detection capabilities and empowers financial institutions to proactively identify and mitigate fraudulent activities. By leveraging the power of Python's set method, we efficiently extract unique event patterns, enabling more accurate analysis and informed decision-making.

Feel free to refer to the provided code and follow along to create a comprehensive understanding of the data fusion process.